In [1]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import wordpunct_tokenize
from nltk import word_tokenize
from nltk.stem.porter import *
from nltk.tokenize import RegexpTokenizer
import os,sys
import re
import pickle
import pandas as pd
import numpy as np
import tqdm
from tqdm import tqdm
import random
import math
from collections import Counter
import functools

In [2]:
with open('data/stop_words.txt') as f:
    stop_words_list = f.read().splitlines() #stop_list1
stop_list2 = pickle.load(open('data/stop_list2.pkl','rb'))
ps = PorterStemmer() # Stemming
stop_words = set(stopwords.words('english')) #Stopword
short = ['.', ',', '"', "\'", '?', '!', ':', ';', '(', ')', '[', ']', '{', '}', "'at",
         "_","`","\'\'","--","``",".,","//",":","___",'_the','-',"'em",".com",
                   '\'s','\'m','\'re','\'ll','\'d','n\'t','shan\'t',"...","\'ve",'u']
stop_words_list.extend(short)
stop_words_list.extend(stop_list2)
stop_words.update(stop_words_list)

In [3]:
def preprocess(texts):
    tokens = [i for i in word_tokenize(texts.lower()) if i not in stop_words]  # Tokenization.# Lowercasing
    token_result = ''
    token_result_ = ''
    for i,token in enumerate(tokens): #list2str
        token_result += ps.stem(token) + ' '
    token_result = ''.join([i for i in token_result if not i.isdigit()])
    token_result = [i for i in word_tokenize(token_result) if i not in stop_words]
    for i,token in enumerate(token_result):
        token_result_ += token + ' '
    return token_result_

## Feature Selection
* tf-idf
* chi-square
* likelihood
* PMI
* EMI

=> build dictionary in 500 words

### LLR

In [380]:
dict_df = pd.read_csv('data/dictionary.txt',header=None,index_col=None,sep=' ')
terms = dict_df[1].tolist() #all terms

In [187]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_dict[trainid[0]] = trainid[1:]
train_dict #class:doc_id
train_dict = pickle.load(open('data/train_dict.pkl','rb'))

{'1': ['11',
  '19',
  '29',
  '113',
  '115',
  '169',
  '278',
  '301',
  '316',
  '317',
  '321',
  '324',
  '325',
  '338',
  '341'],
 '2': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '12',
  '13',
  '14',
  '15',
  '16'],
 '3': ['813',
  '817',
  '818',
  '819',
  '820',
  '821',
  '822',
  '824',
  '825',
  '826',
  '828',
  '829',
  '830',
  '832',
  '833'],
 '4': ['635',
  '680',
  '683',
  '702',
  '704',
  '705',
  '706',
  '708',
  '709',
  '719',
  '720',
  '722',
  '723',
  '724',
  '726'],
 '5': ['646',
  '751',
  '781',
  '794',
  '798',
  '799',
  '801',
  '812',
  '815',
  '823',
  '831',
  '839',
  '840',
  '841',
  '842'],
 '6': ['995',
  '998',
  '999',
  '1003',
  '1005',
  '1006',
  '1007',
  '1009',
  '1011',
  '1012',
  '1013',
  '1014',
  '1015',
  '1016',
  '1019'],
 '7': ['700',
  '730',
  '731',
  '732',
  '733',
  '735',
  '740',
  '744',
  '752',
  '754',
  '755',
  '756',
  '757',
  '759',
  '760'],
 '8': ['262',
  '296',
  '30

In [378]:
in_dir = 'data/IRTM/'
train_dict_ = {}
class_token = []
class_token_dict = {}
for c,d in train_dict.items():
    for doc in d:
        f = open('data/IRTM/'+doc+'.txt')
        texts = f.read()
        f.close()
        tokens_all = preprocess(texts)
        tokens_all = tokens_all.split(' ')
        tokens_all = list(set(filter(None,tokens_all)))
        class_token.append(tokens_all)
    class_token_dict[c]=class_token
    class_token=[]
len(class_token_dict['1'])

52

In [71]:
# train_dict = {}
# for c,t in class_token_dict.items():
#     train_dict[c] = list(set(t))
# #     train_dict[c] = dict(Counter(t))
# train_dict

TypeError: unhashable type: 'list'

In [381]:
dict_df.drop(0,axis=1,inplace=True)
dict_df.columns = ['term','score']
dict_df.index = dict_df['term']
dict_df.drop('term',axis=1,inplace=True)
dict_df

,score
term,
abandon,39
abc,51
abcnews.com,3
abdallah,2
abdel,3
abdomin,2
abduct,2
abdul,53
abdul-karim,11


In [382]:
dict_df['score'] = 0
dict_df['score_chi'] = 0
dict_df['score_emi'] = 0
# c=1
for term in tqdm(terms): #each term
    scores = []
    scores_chi = []
    scores_emi = []
    c=1
    for _ in range(len(class_token_dict)): # each class
        n11=e11=m11=0
        n10=e10=m10=0
        n01=e01=m01=0
        n00=e00=m00=0
        for k,v in class_token_dict.items():
#             print(k,c)
            if k == str(c): #ontopic
                for r in v:
                    if term in r:
                        n11+=1
                    else:
                        n10+=1
#                 c+=1
            else: #off topic
                for r in v:
                    if term in r:
                        n01+=1
                    else:
                        n00+=1
#                 c+=1
        c+=1
        n11+=1e-8
        n10+=1e-8
        n01+=1e-8
        n00+=1e-8
        N = n11+n10+n01+n00
        e11 = N * (n11+n01)/N * (n11+n10)/N #chi-squre
        e10 = N * (n11+n10)/N * (n10+n00)/N
        e01 = N * (n11+n01)/N * (n01+n00)/N
        e00 = N * (n01+n00)/N * (n10+n00)/N
        score_chi = ((n11-e11)**2)/e11 + ((n10-e10)**2)/e10 + ((n01-e01)**2)/e01 + ((n00-e00)**2)/e00
        scores_chi.append(score_chi)
        
        n11 = n11 - 1e-8 + 1e-6
        n10 = n10 - 1e-8 + 1e-6
        n01 = n01 - 1e-8 + 1e-6
        n00 = n00 - 1e-8 + 1e-6
        N = n11+n10+n01+n00
        m11 = (n11/N) * math.log(((n11/N)/((n11+n01)/N * (n11+n10)/N)),2) #EMI
        m10 = n10/N * math.log((n10/N)/((n11+n10)/N * (n10+n00)/N),2)
        m01 = n01/N * math.log((n01/N)/((n11+n01)/N * (n01+n00)/N),2)
        m00 = n00/N * math.log((n00/N)/((n01+n00)/N * (n10+n00)/N),2)
        score_emi = m11 + m10 + m01 + m00
        scores_emi.append(score_emi)
        
#         print(n11,n10,n01,n00)
        n11-=1e-6
        m10-=1e-6
        n01-=1e-6
        n00-=1e-6
        N = n11+n10+n01+n00
        score = (((n11+n01)/N) ** n11) * ((1 - ((n11+n01)/N)) ** n10) * (((n11+n01)/N) ** n01) * ((1 - ((n11+n01)/N)) ** n00)
        score /= ((n11/(n11+n10)) ** n11) * ((1 - (n11/(n11+n10))) ** n10) * ((n01/(n01+n00)) ** n01) * ((1 - (n01/(n01+n00))) ** n00)
        score = -2 * math.log(score, 10) #LLR
        scores.append(score)
        
        
#         c+=1
    dict_df.loc[term,'score'] = np.mean(scores)
    dict_df.loc[term,'score_chi'] = np.mean(scores_chi)
    dict_df.loc[term,'score_emi'] = np.mean(scores_emi)
dict_df    

100%|██████████| 8913/8913 [03:59<00:00, 37.23it/s]


,score,score_chi,score_emi
term,,,
abandon,1.659552,3.846051e+00,2.913791e-03
abc,0.899990,1.555876e+00,1.580176e-03
abcnews.com,0.204845,6.544514e-01,3.596454e-04
abdallah,0.274067,5.290393e-01,4.811811e-04
abdel,0.411467,7.944005e-01,7.224236e-04
abdomin,0.274067,5.290393e-01,4.811811e-04
abduct,0.160386,3.668540e-01,2.815866e-04
abdul,0.464110,9.267195e-01,8.148566e-04
abdul-karim,1.519594,2.937724e+00,2.668045e-03


In [383]:
dict_df2 = pd.read_csv('data/dictionary.txt',header=None,index_col=None,sep=' ')
dict_df2.columns = ['id','term','freq']
dict_df2['sum'] = 0.0
dict_df2

,id,term,freq,sum
0,1,abandon,39,0.0
1,2,abc,51,0.0
2,3,abcnews.com,3,0.0
3,4,abdallah,2,0.0
4,5,abdel,3,0.0
5,6,abdomin,2,0.0
6,7,abduct,2,0.0
7,8,abdul,53,0.0
8,9,abdul-karim,11,0.0
9,10,abdullah,40,0.0


In [384]:
tf_list = next(os.walk('../HW2/output/tf-idf/'))[2]
# df_list = [dict_df2]
for tf in tf_list:
#     print(tf)
    df2 = pd.read_csv('../HW2/output/tf-idf/'+tf,header=None,index_col=None,sep=' ',skiprows=[0])
    df2.columns = ['id','tfidf']
    df3 = pd.merge(dict_df2,df2,on='id',how='outer')
    df3.fillna(0,inplace=True)
    dict_df2['sum']+=df3['tfidf']
dict_df2['avg_tfidf'] = dict_df2['sum']/dict_df2['freq']
dict_df2 = dict_df2.drop(['freq','sum'],axis=1)
dict_df2
#     break
#     df_list.append(df2)
# df3 = pd.concat(df_list).groupby(level=0).sum()
# df3 = pd.concat([df3,df2]).groupby(level=0).sum()
# df3 = pd.merge(dict_df2,df2,on='id',how='outer')
# df3 = pd.merge(df3,df2,on='id',how='outer')?
# df3[df3.id == 2]

,id,term,avg_tfidf
0,1,abandon,0.005374
1,2,abc,0.015857
2,3,abcnews.com,0.033696
3,4,abdallah,0.006597
4,5,abdel,0.003924
5,6,abdomin,0.010949
6,7,abduct,0.016260
7,8,abdul,0.000851
8,9,abdul-karim,0.005135
9,10,abdullah,0.006808


In [385]:
dict_df['term'] = dict_df.index
dict_df3 = pd.merge(dict_df,dict_df2,on='term',how='outer')
dict_df3

/home/leoqaz12/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2961: FutureWarning: 'term' is both an index level and a column label.
Defaulting to column, but this will raise an ambiguity error in a future version
  exec(code_obj, self.user_global_ns, self.user_ns)


,score,score_chi,score_emi,term,id,avg_tfidf
0,1.659552,3.846051e+00,2.913791e-03,abandon,1,0.005374
1,0.899990,1.555876e+00,1.580176e-03,abc,2,0.015857
2,0.204845,6.544514e-01,3.596454e-04,abcnews.com,3,0.033696
3,0.274067,5.290393e-01,4.811811e-04,abdallah,4,0.006597
4,0.411467,7.944005e-01,7.224236e-04,abdel,5,0.003924
5,0.274067,5.290393e-01,4.811811e-04,abdomin,6,0.010949
6,0.160386,3.668540e-01,2.815866e-04,abduct,7,0.016260
7,0.464110,9.267195e-01,8.148566e-04,abdul,8,0.000851
8,1.519594,2.937724e+00,2.668045e-03,abdul-karim,9,0.005135
9,5.383307,1.045027e+01,9.451857e-03,abdullah,10,0.006808


In [386]:
cols = list(dict_df3)
cols[4], cols[3], cols[5], cols[1], cols[0], cols[2] = cols[0], cols[1] , cols[2] , cols[3], cols[4], cols[5]
dict_df3 = dict_df3.ix[:,cols]
dict_df3

/home/leoqaz12/.local/lib/python3.6/site-packages/ipykernel/__main__.py:3: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
  app.launch_new_instance()


,id,term,avg_tfidf,score_chi,score,score_emi
0,1,abandon,0.005374,3.846051e+00,1.659552,2.913791e-03
1,2,abc,0.015857,1.555876e+00,0.899990,1.580176e-03
2,3,abcnews.com,0.033696,6.544514e-01,0.204845,3.596454e-04
3,4,abdallah,0.006597,5.290393e-01,0.274067,4.811811e-04
4,5,abdel,0.003924,7.944005e-01,0.411467,7.224236e-04
5,6,abdomin,0.010949,5.290393e-01,0.274067,4.811811e-04
6,7,abduct,0.016260,3.668540e-01,0.160386,2.815866e-04
7,8,abdul,0.000851,9.267195e-01,0.464110,8.148566e-04
8,9,abdul-karim,0.005135,2.937724e+00,1.519594,2.668045e-03
9,10,abdullah,0.006808,1.045027e+01,5.383307,9.451857e-03


In [387]:
dict_df3.columns = ['id','term','avg_tfidf','score_chi','score_llr','score_emi']
dict_df3.to_csv('output/feature_selection_df_rev.csv')

### select top 500
* 取各col的mean+1.45*std
* 再去做投票，超過兩票的流下來看剩下哪幾個

In [446]:
dict_df3 = pd.read_csv('output/feature_selection_df_rev.csv',index_col=None)
threshold_tfidf = np.mean(dict_df3['avg_tfidf'])+2.5*np.std(dict_df3['avg_tfidf']) #1.45=>502 數字大嚴格
threshold_chi = np.mean(dict_df3['score_chi'])+2.5*np.std(dict_df3['score_chi']) #1=>350
threshold_llr = np.mean(dict_df3['score_llr'])+2.5*np.std(dict_df3['score_llr']) #1.75=>543
threshold_emi = np.mean(dict_df3['score_emi'])+2.5*np.std(dict_df3['score_emi']) #1.75=>543

print('avg_tfidf',threshold_tfidf)
# dict_df3[dict_df3.score_llr>0.1]

avg_tfidf 0.024888109121963365


In [447]:
df1 = dict_df3[dict_df3['avg_tfidf']>threshold_tfidf]
df2 = dict_df3[dict_df3['score_chi']>threshold_chi]
df3 = dict_df3[dict_df3['score_llr']>threshold_llr]
df4 = dict_df3[dict_df3['score_emi']>threshold_emi]
df_vote = dict_df3
df_vote['vote']=0
df_vote

,Unnamed: 0,id,term,avg_tfidf,score_chi,score_llr,score_emi,vote
0,0,1,abandon,0.005374,3.846051e+00,1.659552,2.913791e-03,0
1,1,2,abc,0.015857,1.555876e+00,0.899990,1.580176e-03,0
2,2,3,abcnews.com,0.033696,6.544514e-01,0.204845,3.596454e-04,0
3,3,4,abdallah,0.006597,5.290393e-01,0.274067,4.811811e-04,0
4,4,5,abdel,0.003924,7.944005e-01,0.411467,7.224236e-04,0
5,5,6,abdomin,0.010949,5.290393e-01,0.274067,4.811811e-04,0
6,6,7,abduct,0.016260,3.668540e-01,0.160386,2.815866e-04,0
7,7,8,abdul,0.000851,9.267195e-01,0.464110,8.148566e-04,0
8,8,9,abdul-karim,0.005135,2.937724e+00,1.519594,2.668045e-03,0
9,9,10,abdullah,0.006808,1.045027e+01,5.383307,9.451857e-03,0


In [448]:
df_vote.loc[df1.id-1,'vote'] += 1
df_vote.loc[df2.id-1,'vote'] += 1
df_vote.loc[df3.id-1,'vote'] += 1
df_vote.loc[df4.id-1,'vote'] += 1
# df_vote

In [449]:
df_vote_ = df_vote[df_vote.vote>2] #(1,2)=>375 #(1,1)=>422 #(1.6,2)=>482 #(2,2)=>330 #(1,3)=>100
df_vote_ = df_vote_.filter(['id','term','vote'])
df_vote_

,id,term,vote
12,13,abidjan,3
81,82,aden,3
130,131,african,3
134,135,aftershock,3
136,137,agassi,4
202,203,alassan,3
208,209,alberto,3
310,311,andr,3
385,386,appoint,3
396,397,april,3


In [450]:
df_vote_.to_csv('output/500terms_df_rev5.csv')

## Classifier
* 7-fold
* MNB
* BNB
* self-train / co-train
* ens voting (BNB lower weight)
* auto-skleran / auto-keras

REF: http://kenzotakahashi.github.io/naive-bayes-from-scratch-in-python.html

In [451]:
df_vote = pd.read_csv('output/500terms_df_rev5.csv',index_col=False)
terms_li = list(set(df_vote.term.tolist()))

train_X = []
train_Y = []
len(terms_li)

186

In [452]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}

for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_dict[trainid[0]] = trainid[1:]
# train_dict #class:doc_id
train_dict = pickle.load(open('data/train_dict.pkl','rb'))
in_dir = 'data/IRTM/'
train_dict_ = {}
class_token = []
class_token_dict = {}
train_X = []
train_Y= []
train_ids = []
for c,d in tqdm(train_dict.items()):
    for doc in d:
        train_ids.append(doc)
        trainX = np.array([0]*len(terms_li))
        f = open('data/IRTM/'+doc+'.txt')
        texts = f.read()
        f.close()
        tokens_all = preprocess(texts)
        tokens_all = tokens_all.split(' ')
#         tokens_all = list(filter(None,tokens_all))
        tokens_all = dict(Counter(tokens_all))
        for key,value in tokens_all.items():
            if key in terms_li:
                trainX[terms_li.index(key)] = int(value)
#         trainX = np.array(trainX)
        
#         for token in tokens_all:
#             if token in terms_li:
#                 ind = terms_li.index(token)
#                 trainX[ind]+=1
        train_X.append(trainX)
        train_Y.append(int(c))
        
train_X = np.array(train_X)
train_Y = np.array(train_Y)

#         tokens_all = list(set(filter(None,tokens_all)))
#         class_token.append(tokens_all)
#     class_token_dict[c]=class_token
#     class_token=[]
# len(class_token_dict['1'])
print(train_X.shape , train_Y.shape)

100%|██████████| 13/13 [00:07<00:00,  1.78it/s]

(946, 186) (946,)


In [254]:
#建立term index matrix
tokens_all_class=[]
term_tf_mat=[]
for c,d in tqdm(train_dict.items()):
    for doc in d:
        f = open('data/IRTM/'+doc+'.txt')
        texts = f.read()
        f.close()
        tokens_all = preprocess(texts)
        tokens_all = tokens_all.split(' ')
        tokens_all = list(filter(None,tokens_all))
        tokens_all_class.extend(tokens_all)
    tokens_all = dict(Counter(tokens_all_class))
    term_tf_mat.append(tokens_all)

100%|██████████| 13/13 [00:01<00:00,  6.61it/s]


In [255]:
def train_MNB(train_set=train_dict,term_list=terms_li,term_tf_mat=term_tf_mat):
    prior = np.zeros(len(train_set))
    cond_prob = np.zeros((len(train_set), len(term_list)))
    
    for i,docs in train_set.items(): #13 classes 1~13
        prior[int(i)-1] = len(docs)/len(train_ids) #那個類別的文章有幾個/總共的文章數目 0~12
        token_count=0
        class_tf = np.zeros(len(term_list))
        for idx,term in enumerate(term_list):
            try:
                class_tf[idx] = term_tf_mat[int(i)-1][term]  #term在class的出現次數
            except:
                token_count+=1

        class_tf = class_tf + np.ones(len(term_list)) #smoothing (可改)
        class_tf = class_tf/(sum(class_tf) +token_count) #該class總共的token數(可改)
        cond_prob[int(i)-1] = class_tf #0~12
    return prior, cond_prob

In [256]:
prior,cond_prob = train_MNB()
prior

array([0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308,
       0.07692308, 0.07692308, 0.07692308, 0.07692308, 0.07692308,
       0.07692308, 0.07692308, 0.07692308])

In [257]:
def predict_MNB(test_id,prob=False,prior=prior,cond_prob=cond_prob,term_list=terms_li):
    f = open('data/IRTM/'+str(test_id)+'.txt')
    texts = f.read()
    f.close()
    tokens_all = preprocess(texts)
    tokens_all = tokens_all.split(' ')
    tokens_all = list(filter(None,tokens_all))
    
    class_scores = []
#     score = 0
    for i in range(13):
        score=0
#         print(prior[i])
        score += math.log(prior[i],10)
        for token in tokens_all:
            if token in term_list:
                score += math.log(cond_prob[i][term_list.index(token)])
        class_scores.append(score)
    if prob:
        return np.array(class_scores)
    else:
        return(np.argmax(class_scores)+1)

In [261]:
ans = predict_MNB(20,prob=False) # 17 18 20 21
ans

2

In [235]:
ans=[]
for i in tqdm(range(1095)):
    ans.append(predict_MNB(i+1))

  0%|          | 0/1095 [00:00<?, ?it/s]


ValueError: math domain error

In [232]:
# np.max(ans)
ans

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [80]:
def MNB(input_X,input_Y=None,prior_log_class=None,log_prob_feature=None,train=True,prob=False,smooth=1.0):
    if train:
        sample_num = input_X.shape[0]
        match_data = [[x for x, t in zip(input_X, input_Y) if t == c] for c in np.unique(input_Y)]
        prior_log_class = [np.log(len(i) / sample_num) for i in match_data]
        counts = np.array([np.array(i).sum(axis=0) for i in match_data]) + smooth
        log_prob_feature = np.log(counts / counts.sum(axis=1)[np.newaxis].T)
        return prior_log_class,log_prob_feature
    else:
        probability = [(log_prob_feature * x).sum(axis=1) + prior_log_class for x in input_X]
        if prob:
            return probability
        else:
            ans = np.argmax(probability,axis=1)
            return ans

In [18]:
class BernoulliNB(object):
    def __init__(self, alpha=1.0, binarize=0.0):
        self.alpha = alpha
        self.binarize = binarize
    def _binarize_X(self, X):
        return np.where(X > self.binarize, 1, 0) if self.binarize != None else X
    def fit(self, X, y):
        X = self._binarize_X(X)
        count_sample = X.shape[0]
        separated = [[x for x, t in zip(X, y) if t == c] for c in np.unique(y)]
        self.class_log_prior_ = [np.log(len(i) / count_sample) for i in separated]
        count = np.array([np.array(i).sum(axis=0) for i in separated]) + self.alpha
        smoothing = 2 * self.alpha
        n_doc = np.array([len(i) + smoothing for i in separated])
        self.feature_prob_ = count / n_doc[np.newaxis].T
        return self

    def predict_log_proba(self, X):
        X = self._binarize_X(X)
        return [(np.log(self.feature_prob_) * x + \
                 np.log(1 - self.feature_prob_) * np.abs(x - 1)
                ).sum(axis=1) + self.class_log_prior_ for x in X]

    def predict(self, X):
        X = self._binarize_X(X)
        return np.argmax(self.predict_log_proba(X), axis=1)

In [296]:
X = np.array([
    [2,1,0,0,0,0],
    [2,0,1,0,0,0],
    [1,0,0,1,0,0],
    [1,0,0,0,1,1]
])
y = np.array([0,1,2,3])
nb = BernoulliNB().fit(X, y)
X_test = np.array([[3,0,0,0,1,1],[0,1,1,0,1,1],[1,0,0,0,1,1],[2,1,0,0,0,0],[2,0,1,0,0,0],[1,0,0,1,0,0]])
print(nb.predict_log_proba(X_test))

[array([-9.07658038, -9.07658038, -9.70406053, -8.90668135]), array([-9.48204549, -9.48204549, -9.70406053, -8.78889831]), array([-6.8793558 , -6.8793558 , -6.93147181, -5.89852655]), array([-5.08759634, -5.78074352, -6.23832463, -6.59167373]), array([-5.78074352, -5.08759634, -6.23832463, -6.59167373]), array([-4.68213123, -4.68213123, -4.15888308, -5.08759634])]


### Prediction

In [453]:
df_vote = pd.read_csv('output/500terms_df_rev5.csv',index_col=False)
terms_li = list(set(df_vote.term.tolist()))
len(terms_li)

186

In [263]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
test_id = []
train_ids=[]
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_ids.extend(trainid[1:])
for i in range(1095):
    if str(i+1) not in train_ids:
        test_id.append(i+1)
ans=[]
for doc in tqdm(test_id):
    ans.append(predict_MNB(doc))
print(ans)
df_ans = pd.DataFrame(list(zip(test_id,ans)),columns=['id','Value'])
df_ans.to_csv('output/MNB.csv',index=False)
df_ans

100%|██████████| 900/900 [00:20<00:00, 43.87it/s]


[2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 13,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 3,
 3,
 2,
 2,
 2,
 2,
 3,
 4,
 3,
 3,
 4,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 3,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 13,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 13,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 2,
 12,
 13,
 13,
 2,
 2,
 10,
 2,
 2,
 13,
 9,
 9,
 9,
 9,
 11,
 9,
 9,
 9,
 11,
 9,
 9,
 13,
 9,
 9,
 9,
 9,
 11,
 9,
 9,
 9,
 13,
 9,
 10,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 13,
 9,
 9,
 13,
 9,
 10,
 9,
 2,
 9,
 9,
 13,
 9,
 9,
 13,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 13,
 11,
 13,
 11,
 9,
 13,
 9,
 11,
 9,
 9,
 9,
 9,
 10,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 9,
 2,
 9,
 9,
 9,
 9,
 9,
 2,
 9,
 9,
 9,
 9,
 9,
 13,
 13,
 13,
 9,
 9,
 9,
 9,
 11,
 9,
 9,
 2,
 9,
 9,
 11,
 11,
 9,
 9,
 9,
 10,
 9,
 11,
 9,
 9,
 13,
 9,
 9,
 10,
 10,
 10,
 9,
 9,
 9,
 9,
 9,
 13,
 9,
 9,
 9,
 11,
 9,
 1,
 11,
 9,
 9,
 11,
 9,
 9

combine all prediction df

In [458]:
import os
in_dir = './output/'
prefixed = [filename for filename in os.listdir('./output/') if filename.endswith("_sk.csv")]
df_from_each_file = [pd.read_csv(in_dir+f) for f in prefixed]
prefixed

['MNB02_sk.csv',
 'BNB01_sk.csv',
 'MNB01_sk.csv',
 'MNB05_sk.csv',
 'MNB04_sk.csv',
 'BNB03_sk.csv',
 'MNB03_sk.csv',
 'BNB04_sk.csv',
 'BNB02_sk.csv']

In [459]:
merged_df = functools.reduce(lambda left,right: pd.merge(left,right,on='id'), df_from_each_file)
merged_df.columns = ['id',0,1,2,3,4,5,6,7,8]
merged_df

,id,0,1,2,3,4,5,6,7,8
0,17,2,2,2,2,2,2,2,2,2
1,18,2,2,2,2,2,2,2,2,2
2,20,2,2,2,2,2,2,2,2,2
3,21,2,2,2,2,2,2,2,2,2
4,22,2,2,2,2,2,2,2,2,9
5,23,2,2,2,2,2,2,2,2,2
6,24,2,2,2,2,2,2,2,2,2
7,25,2,2,2,2,2,2,2,2,2
8,26,2,2,2,2,2,2,9,2,9
9,27,2,2,2,2,2,2,2,2,2


In [477]:
df01 = pd.DataFrame(merged_df.mode(axis=1)[0])
df02 = pd.DataFrame(merged_df['id'])
df_ans = pd.concat([df02,df01],axis=1)
df_ans = df_ans.astype('int')
df_ans.columns = ['id','Value']
df_ans.to_csv('output/voting_rev.csv',index=False)
df_ans

,id,Value
0,17,2
1,18,2
2,20,2
3,21,2
4,22,2
5,23,2
6,24,2
7,25,2
8,26,2
9,27,2


In [359]:
df1 = merged_df[(merged_df[0] == merged_df[1])&(merged_df[2]==merged_df[3])&(merged_df[1]==merged_df[2])]
df1.reset_index(inplace=True,drop=True)
df1['class'] = df1[0]
df1 = df1.filter(['id','class'])
df1

/home/leoqaz12/.local/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


,id,class
0,17,2
1,18,2
2,20,2
3,21,2
4,23,2
5,24,2
6,25,2
7,27,2
8,30,2
9,33,2


In [375]:
df1[df1['class']=='1']

,id,class
236,357,1
245,369,1
248,372,1
253,377,1
257,381,1
259,383,1
260,384,1
262,386,1
264,388,1
265,389,1


In [373]:
for i in range(13):
    df1 = df1.astype(str)
    li = df1[df1['class'] == str(i+1)]['id'].tolist()
    train_dict[str(i+1)].extend(li)
train_dict

{'1': ['11',
  '19',
  '29',
  '113',
  '115',
  '169',
  '278',
  '301',
  '316',
  '317',
  '321',
  '324',
  '325',
  '338',
  '341',
  '357',
  '369',
  '372',
  '377',
  '381',
  '383',
  '384',
  '386',
  '388',
  '389',
  '396',
  '400',
  '402',
  '405',
  '419',
  '422',
  '423',
  '429',
  '431',
  '435',
  '444',
  '451',
  '460',
  '464',
  '467',
  '468',
  '476',
  '477',
  '479',
  '482',
  '489',
  '494',
  '524',
  '543',
  '550',
  '625',
  '811'],
 '2': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '12',
  '13',
  '14',
  '15',
  '16',
  '17',
  '18',
  '20',
  '21',
  '23',
  '24',
  '25',
  '27',
  '30',
  '33',
  '34',
  '35',
  '38',
  '39',
  '40',
  '42',
  '43',
  '45',
  '47',
  '48',
  '49',
  '50',
  '51',
  '53',
  '54',
  '55',
  '56',
  '57',
  '58',
  '59',
  '60',
  '61',
  '62',
  '64',
  '65',
  '66',
  '68',
  '69',
  '71',
  '72',
  '73',
  '74',
  '75',
  '76',
  '77',
  '78',
  '79',
  '80',
  '81',
  '82',
  '84',
  '85

In [377]:
pickle.dump(obj=train_dict,file=open('data/train_dict.pkl','wb'))

In [372]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_dict[trainid[0]] = trainid[1:]
train_dict #class:doc_id

{'1': ['11',
  '19',
  '29',
  '113',
  '115',
  '169',
  '278',
  '301',
  '316',
  '317',
  '321',
  '324',
  '325',
  '338',
  '341'],
 '2': ['1',
  '2',
  '3',
  '4',
  '5',
  '6',
  '7',
  '8',
  '9',
  '10',
  '12',
  '13',
  '14',
  '15',
  '16'],
 '3': ['813',
  '817',
  '818',
  '819',
  '820',
  '821',
  '822',
  '824',
  '825',
  '826',
  '828',
  '829',
  '830',
  '832',
  '833'],
 '4': ['635',
  '680',
  '683',
  '702',
  '704',
  '705',
  '706',
  '708',
  '709',
  '719',
  '720',
  '722',
  '723',
  '724',
  '726'],
 '5': ['646',
  '751',
  '781',
  '794',
  '798',
  '799',
  '801',
  '812',
  '815',
  '823',
  '831',
  '839',
  '840',
  '841',
  '842'],
 '6': ['995',
  '998',
  '999',
  '1003',
  '1005',
  '1006',
  '1007',
  '1009',
  '1011',
  '1012',
  '1013',
  '1014',
  '1015',
  '1016',
  '1019'],
 '7': ['700',
  '730',
  '731',
  '732',
  '733',
  '735',
  '740',
  '744',
  '752',
  '754',
  '755',
  '756',
  '757',
  '759',
  '760'],
 '8': ['262',
  '296',
  '30

In [328]:
df_from_each_file = [pd.read_csv(in_dir+f) for f in prefixed]
# concatenated_df   = pd.concat(df_from_each_file,axis=1,join='inner')
concatenated_df   = pd.DataFrame.join(df_from_each_file,on='id')
concatenated_df

TypeError: join() missing 1 required positional argument: 'other'

In [319]:
df1 = pd.read_csv(in_dir+prefixed[0])
df2 = pd.read_csv(in_dir+prefixed[1])
pd.merge(df1,df2,on='id',how='inner')

TypeError: merge() missing 1 required positional argument: 'right'

In [454]:
with open('data/training.txt','r') as f:
    train_id = f.read().splitlines()
train_dict = {}
test_id = []
train_ids=[]
for trainid in train_id:
    trainid = trainid.split(' ')
    trainid = list(filter(None, trainid))
    train_ids.extend(trainid[1:])
for i in range(1095):
    if str(i+1) not in train_ids:
        test_id.append(i+1)
#     train_dict[trainid[0]] = trainid[1:]
# train_dict #class:doc_id
in_dir = 'data/IRTM/'
train_dict_ = {}
class_token = []
class_token_dict = {}
test_X = []
# train_Y= []
# for c,d in tqdm(train_dict.items()):
for doc in tqdm(test_id):
    testX = np.array([0]*len(terms_li))
    f = open('data/IRTM/'+str(doc)+'.txt')
    texts = f.read()
    f.close()
    tokens_all = preprocess(texts)
    tokens_all = tokens_all.split(' ')
#         tokens_all = list(filter(None,tokens_all))
    tokens_all = dict(Counter(tokens_all))
    for key,value in tokens_all.items():
        if key in terms_li:
            testX[terms_li.index(key)] = int(value)

    test_X.append(testX)
        
test_X = np.array(test_X)
print(test_X.shape)

100%|██████████| 900/900 [00:06<00:00, 131.03it/s]

(900, 186)


In [113]:
df_ans = pd.DataFrame(list(zip(test_id,ans2)),columns=['id','Value'])
df_ans.to_csv('output/MNB02_sk.csv',index=False)
df_ans

,id,Value
0,17,2
1,18,2
2,20,2
3,21,2
4,22,2
5,23,2
6,24,2
7,25,2
8,26,2
9,27,2


In [127]:
from sklearn.datasets import load_iris
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

In [291]:
from sklearn.naive_bayes import *
clf = MultinomialNB()
clf.fit(train_X, train_Y)
ans2 = clf.predict(test_X)
ans2

array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2, 11,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2, 11,  2,  2,  2,  2,  2, 13,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  2,  9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  2,  9,  9,  9,  9,  9,
        2,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  2,
        9,  9,  9,  9,  9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9

In [456]:
from sklearn.naive_bayes import *
clf = MultinomialNB()
clf.fit(train_X, train_Y)
ans2 = clf.predict(test_X)
ans2

array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2, 11,  9,
        9,  9,  9, 10,  9,  9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,
        9,  9, 13,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  2,  9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  2,  9,  9,  9,  9,  9,
        2,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  2,
        9,  9,  9,  9,  9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9

In [444]:
clf = BernoulliNB()
clf.fit(train_X, train_Y)
ans2 = clf.predict(test_X)
ans2

array([ 2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,
        2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  2,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9, 11,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  2,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  2,  9,  9,  9,  9,  9,
        2,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  2,
        9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,  9,
        9,  9,  9,  9,  9

In [455]:
print(train_X.shape, train_Y.shape , test_X.shape)

(946, 186) (946,) (900, 186)


In [457]:
df_ans = pd.DataFrame(list(zip(test_id,ans2)),columns=['id','Value'])
df_ans.to_csv('output/MNB05_sk.csv',index=False)
df_ans

,id,Value
0,17,2
1,18,2
2,20,2
3,21,2
4,22,2
5,23,2
6,24,2
7,25,2
8,26,2
9,27,2
